In [1]:
from db.client import PGClient
from collections import ChainMap
from datetime import datetime
import re
import emoji
import pandas as pd
client = PGClient()

In [7]:
uni_emojis_list = map(lambda x: ''.join(x.split()), emoji.UNICODE_EMOJI.keys())
uni_emojis_pt = re.compile('|'.join(re.escape(p) for p in uni_emojis_list))

def clean(c):
    url_pt = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    custom_emoji_pt = '<:\w+:[0-9]+>'
    tag_pt = '<@!?[0-9]+>'
   
    c = str(c)
    c = re.sub(url_pt, '', c)
    c = re.sub(tag_pt, '', c)
    c = re.sub('\n',' ', c)
    c = uni_emojis_pt.sub('', c)
    c = re.sub('"','', c)
    
    emojis = re.findall(custom_emoji_pt,c)
    for e in emojis:
        e_new = e.split(':')[1].lower()
        c = re.sub(e, '', c)

    return c.strip()

In [114]:
def get_messages(first_id=None, last_id=None, by_author=False):
    q_messages = """SELECT json_build_object('id',id,'posted_at',posted_at,
                                             'content',content, 'author_name', author_name,
                                             'author_id', author_id)
                    FROM messages 
                    WHERE author_id NOT IN (155149108183695360, 349547918966915073, 159985870458322944) """
    if first_id:
        q_messages += "AND id >= " + str(first_id)
    if last_id:
        q_messages += "AND id <= " + str(last_id) 
    
    q_messages += "ORDER BY id"

    cursor = client.query(q_messages)
    message = cursor.fetchone()

    if by_author:
        messages = {}
        while message:
            message = message[0]
            author_id = str(message['author_id'])
            del message['author_id']
            del message['author_name']
            if author_id in messages:
                messages[author_id].append(message)
            else:
                messages[author_id] = [message]
            message = cursor.fetchone()
    else:
        messages = []
        while message:
            #message = message[0]
            #message['content'] = clean(message['content'])
            #if message['content']:
            messages.append(message[0])

            message = cursor.fetchone()
    
    return messages

In [31]:
def get_discussions(min_char_len=250, max_time_diff=30):
    q_messages = """SELECT json_build_object('id',id,'posted_at',posted_at,'content',content, 'author_name', author_name) FROM messages 
                    WHERE author_id NOT IN (155149108183695360, 349547918966915073, 159985870458322944)
                    ORDER BY id
                    """ 

    cursor = client.query(q_messages)
    message = cursor.fetchone()
    discussions = [[]]
    previous_message = datetime.strptime(message[0]['posted_at'], '%Y-%m-%dT%H:%M:%S')

    while message:
        message = message[0]
        content = clean(message['content'])
        message_date = datetime.strptime(message['posted_at'], '%Y-%m-%dT%H:%M:%S')
        if content:
            message['content'] = content
            time_diff = (message_date - previous_message).total_seconds() / 60
            if(time_diff > max_time_diff):
                discussions.append([message])

            else:
                discussions[-1:][0].append(message)
        previous_message = message_date
        message = cursor.fetchone()

    contents = []
    for d in discussions:
        if len(d) > min_char_len:
            content = ''
            for message in d:
                content += ' ' + message['content']
            contents.append(content)

    return contents

In [38]:
def extract_content(messages):
    content = ''
    for message in messages:
        content += ' ' + message['content']

    return content

In [136]:
import nltk
import re
def split_content(content, n):
    i = 0
    c = ''
    contents = []
    for w in content.split(' '):
        if (i + len(w)) <= n:
            c += w + ' '
            i += len(w)
        else:
            contents.append(re.sub('\s+', ' ', c).strip())
            c = w + ' '
            i = len(w)
    contents.append(re.sub('\s+', ' ', c).strip())
        
    return contents

In [82]:
d_trela = get_messages(372759366392348683, 372782278138396673)
d_febo = get_messages(373118183298826243, 373157096436400131)
d_blade = get_messages(389492652934168576, 389536301671776258)
d_radek = get_messages(350181842198200321, 350235040942391308)

In [83]:
n = 3000
d_trela = split_content(clean(extract_content(d_trela)), n)
d_febo = split_content(clean(extract_content(d_febo)), n)
d_blade = split_content(clean(extract_content(d_blade)), n)
d_radek = split_content(clean(extract_content(d_radek)), n)

In [84]:
print(len(d_trela))
print(len(d_febo))
print(len(d_blade))
print(len(d_radek))

4
2
3
3


In [118]:
d_trela_a = get_messages(372759366392348683, 372782278138396673, by_author=True)
d_febo_a = get_messages(373118183298826243, 373157096436400131, by_author=True)
d_blade_a = get_messages(389492652934168576, 389536301671776258, by_author=True)
d_radek_a = get_messages(350181842198200321, 350235040942391308, by_author=True)

In [121]:
print(len(d_trela_a.keys()))
print(len(d_febo_a.keys()))
print(len(d_blade_a.keys()))
print(len(d_radek_a.keys()))

12
10
10
9


In [140]:
merged = {}
for a, d in d_trela_a.items():
    ab = split_content(clean(extract_content(d)), n)
    merged[a] = ab


In [153]:
len(merged['366237556280393728'][0])

3645

In [152]:
n

3000

In [146]:
merged

{'156052059555233792': ["Drama Drama happend Basically Storm accidentally put his dick pic Xan replied with a pic with a cum With a cum Fignore trelabot Ik Funigore trelacki Im back U are mentally disabled. Understandable kjx sir In a pretty way yes Kubi emote I have idea Kubi = MingLee 2⃣ 3⃣ plot twist i get promoted to 99 izno pm basem use VPN germany and dont talk no drama then :gg: or that ah limited nice picture *sigh* *unzips* here we go again yes radek priest so i need to know am i getting demoted or no? i believe the logic behind this is if you are higher level than usual (60+), you are suppose to be nice to people and helping them, etc. not being racist OR hurting their feelings. because if you are 60+ you are trusted with a server control therefore there are people who are trusting you. honestly. radek. NO OFFEENSE PLEASE I STILL LOVE YOU. but you shouldn't be 90 IMHO because you are being mean to basem etc. which is clearly not allowed what if there are other tunisian,iraq, 

In [5]:
df = pd.DataFrame(d_contents)
df.to_clipboard(index=False, header=False)

In [75]:
from paralleldots import set_api_key, get_api_key, sentiment, similarity, emotion

In [76]:
set_api_key("iQVOCSTVfHzRd67ybpvqaEWEgCRQefqWuIQfRypVm0I") 


In [79]:
discussions = [d_trela, d_febo, d_blade, d_radek]
sentiments = []
emotions = []
for d in discussions:
    d_sentiments = []
    d_emotions = []
    for d_part in d:
        #d_sentiments.append(sentiment(d_part))
        d_emotions.append(emotion(d_part))
    #sentiments.append(d_sentiments)
    emotions.append(d_emotions)

In [80]:
emotions

[[{'emotion': 'indifferent',
   'probabilities': {'angry': 0.016536,
    'excited': 0.147496,
    'happy': 0.090375,
    'indifferent': 0.692472,
    'sad': 0.05312},
   'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'},
  {'emotion': 'angry',
   'probabilities': {'angry': 0.761599,
    'excited': 0.001831,
    'happy': 0.002428,
    'indifferent': 0.034183,
    'sad': 0.19996},
   'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'},
  {'emotion': 'indifferent',
   'probabilities': {'angry': 0.194818,
    'excited': 0.055348,
    'happy': 0.025928,
    'indifferent': 0.36921,
    'sad': 0.354696},
   'usage': 'By accessing ParallelDots API or using information generated by 

In [78]:
sentiments

[[{'probabilities': {'negative': 0.036359,
    'neutral': 0.849968,
    'positive': 0.113673},
   'sentiment': 'neutral',
   'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'},
  {'probabilities': {'negative': 0.967067,
    'neutral': 0.029135,
    'positive': 0.003799},
   'sentiment': 'negative',
   'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'},
  {'probabilities': {'negative': 0.458571,
    'neutral': 0.51089,
    'positive': 0.030539},
   'sentiment': 'neutral',
   'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-condit

In [8]:
n = 1000
[s[i:i+n] for i in range(0, len(s), n)]

['Drama what happend? Drama happend hello tomis radek made storm rage how? spit on my smiling balls pls Basically Storm accidentally put his dick pic Xan replied with a pic with a cum with a cum With a cum with the thing You put in levcakes. Fignore trelabot with a glue lev god left You permamently Ik seenable type funignore trelabot to bring him back Funigore trelacki god is back Hi lev Im back I thought lev fignored me cause i couldnt see him in freespec turned out i actually fignored him U are mentally disabled. Understandable kjx sir You mean his disease is retard? XD someone make a :kubi:  emote In a pretty way yes Kubi emote I have idea kubi emote.. screen of writing TESLA with colors on chat? XD or iznodolf? ;D :23: :23: XD hahaaha Kubi = MingLee 2⃣ 3⃣ kay going to geek classic tetris as I wait for dinner brb jm_tetris? nope new pic, you can jerk it off to me WHERE IS MY NEW PIC KURWEA there is lemme smash are u working on your map? no no mood for that im chilling watching sad v

In [10]:
sentiment(s[0]+s[1])

{'probabilities': {'negative': 0.150837,
  'neutral': 0.661699,
  'positive': 0.187464},
 'sentiment': 'neutral',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [11]:
sentiment(s[0]+s[1]+s[2])

{'probabilities': {'negative': 0.195153,
  'neutral': 0.545435,
  'positive': 0.259412},
 'sentiment': 'neutral',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [27]:
emotion(s[1]+s[2]+s[3] + s[4] + s[5] + s[6] + s[7] + s[8] + s[9])

{'emotion': 'indifferent',
 'probabilities': {'angry': 0.060992,
  'excited': 0.248404,
  'happy': 0.240541,
  'indifferent': 0.374858,
  'sad': 0.075205},
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [24]:
sentiment(s[1]+s[2]+s[3] + s[4] + s[5] + s[6] + s[7] + s[8] + s[9] + s[10])

{'probabilities': {'negative': 0.218834,
  'neutral': 0.624937,
  'positive': 0.156229},
 'sentiment': 'neutral',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [63]:
s = "are terrorists? fucking special code to detonate bomb or what what if there are other tunisian,iraq, etc. people on server do you think they will take it as a joke? I fignore them lol and what do they do to offend you? they exist? they believe in killing people? you cant know what a random guy believes in if he is from iraq, he is muslim if he is muslim, he believe in killing people Trela hell no fuck trela i really need AN OFFICIAL statement thats because all christians wait till marriage to have sex. of what you just said maybe if he is member of ISIS limited go to orban and tell him they are cool and you want 1 in your house You know nothign about orban ill go to fucking orban and tell him i want one in my server k xDDDD Orban hates them Quran Quran (2:191-193) - And kill them wherever you find them, and turn them out from where they have turned you out. And Al-Fitnah [disbelief or unrest] is worse than killing... but if they desist, then lo! Allah is forgiving and merciful. And fight them until there is no more Fitnah [disbelief and worshipping of others along with Allah] and worship is for Allah alone But if they cease, let there be no transgression except against Az-Zalimun(the polytheists, and wrong-doers, etc.) (Translation is from the Noble Quran) The verse prior to this (190) refers to fighting for the cause of Allah those who fight you leading some to claim that the entire passage refers to a defensive war in which Muslims are defending their homes and families.  The historical context of this passage is not defensive warfare, however, since Muhammad and his Muslims had just relocated to Medina and were not under attack by their Meccan adversaries.  In fact, the verses urge offensive warfare, in that Muslims are to drive Meccans out of their own city (which they later did).  Verse 190 thus means to fight those who offer resistance to Allah's rule (ie. Muslim conquest).  The use of the word persecution by some Muslim translators is disingenuous - the actual Arabic words for persecution (idtihad) - and oppression are not used instead of fitna.  Fitna can mean disbelief, or the disorder that results from unbelief or temptation.  A strict translation is 'sedition,' meaning rebellion against authority (the authority being Allah).  This is certainly what is meant in this context since the violence is explicitly commissioned until religion is for Allah - ie. unbelievers desist in their unbelief.  [Editor's note: these notes have been modified slightly after a critic misinterpreted our language. Verse 193 plainly says that 'fighting' is sanctioned even if the fitna 'ceases'.  This is about religious order, not real persecution.] have fun hitler was atheist. ANd a good painter i dont need to quote his ideology here i hope. ye, and You love them limited, so go for a conversation with him you're basically quoting bible verses here trela. Who said I love the terrorists? which are about as bad as"
sentiment(s)

{'probabilities': {'negative': 0.771397,
  'neutral': 0.215016,
  'positive': 0.013588},
 'sentiment': 'negative',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [69]:
s = "are terrorists? fucking special code to detonate bomb or what what if there are other tunisian,iraq, etc. people on server do you think they will take it as a joke? I fignore them lol and what do they do to offend you? they exist? they believe in killing people? you cant know what a random guy believes in if he is from iraq, he is muslim if he is muslim, he believe in killing people Trela hell no fuck trela i really need AN OFFICIAL statement thats because all christians wait till marriage to have sex. of what you just said maybe if he is member of ISIS limited go to orban and tell him they are cool and you want 1 in your house You know nothign about orban ill go to fucking orban and tell him i want one in my server k xDDDD Orban hates them Quran Quran (2:191-193) - And kill them wherever you find them, and turn them out from where they have turned you out. And Al-Fitnah [disbelief or unrest] is worse than killing... but if they desist, then lo! Allah is forgiving and merciful. And fight them until there is no more Fitnah [disbelief and worshipping of others along with Allah] and worship is for Allah alone. But if they cease, let there be no transgression except against Az-Zalimun(the polytheists, and wrong-doers, etc.) (Translation is from the Noble Quran) The verse prior to this (190) refers to fighting for the cause of Allah those who fight you leading some to claim that the entire passage refers to a defensive war in which Muslims are defending their homes and families.  The historical context of this passage is not defensive warfare, however, since Muhammad and his Muslims had just relocated to Medina and were not under attack by their Meccan adversaries.  In fact, the verses urge offensive warfare, in that Muslims are to drive Meccans out of their own city (which they later did).  Verse 190 thus means to fight those who offer resistance to Allah's rule (ie. Muslim conquest).  The use of the word persecution by some Muslim translators is disingenuous - the actual Arabic words for persecution (idtihad) - and oppression are not used instead of fitna.  Fitna can mean disbelief, or the disorder that results from unbelief or temptation.  A strict translation is 'sedition,' meaning rebellion against authority (the authority being Allah).  This is certainly what is meant in this context since the violence is explicitly commissioned until religion is for Allah - ie. unbelievers desist in their unbelief.  [Editor's note: these notes have been modified slightly after a critic misinterpreted our language. Verse 193 plainly says that 'fighting' is sanctioned even if the fitna 'ceases'.  This is about religious order, not real persecution.] have fun hitler was atheist. ANd a good painter i dont need to quote his ideology here i hope. ye, and You love them limited, so go for a conversation with him you're basically quoting bible verses here trela. Who said I love the terrorists? which are about as bad as quran ones there is a big space between hating and loving trela. I think Im out No point of this argument izno never shared his information what if hes a muslim? Trela is full of hate, so no point you still talk to him i believe those words are out of context..real muslims are friendly and not monsters..but ye this is what media presents us trela has no way of knowing my religion. read this shit book claay then simple. Quran (2:191-193) Have u ever read it? Quran (2:191-193) Quran (2:191-193) Quran (2:191-193) google it yes 3 verses thats all u got GO READ IT THEN TALK TO ME the worst guy from lebanon that i know is a fucking christian GET SOME FKN AKNOWLEDGE the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. ABOUT THIS DISEASE do you believe in god trela? Hard to not belive in yourself lol. believe* trela u didnt notes that u talk too much about muslims and say some shit about them ? i am serious do you believe in the christian god trela? do you believe in real God. christian god yes. Using those types of stories without context would be like saying that the United States is a racist/genocidal country because they bombed Japan in 1945, killing 250,000 people. The statement that they bombed Japan is true, but there's a whole context (a preceding global war that lasted 6 years) that it needs to be considered in the context of. how can I believe in 1 person which was walking on the water Izno please that was his son you dont even know christian religion how can you judge muslims so you dont believe in christian god BUT you believe in Quran which is basically an islam bible cause I don't care about this religion I don't believe in this, these retards believe that's why they kill people GEEZOS one min son of god ? wtf lol basem, as a muslim, do you belive in killing people? as a muslim k ISIS presents ISLAM in their way . and thats what u see thats what media wants us to believe. thats not even islam its disgreace im giving up on trela. he really wants to see this stuff his way, and there is no way to change his mind, no matter how much evidence to the contrary you present him. and right now he'll be convinced that we are all pro-islam becuase we sound anti-trela you see it right Izno ^ pls islam word mean security, safety, peace, belly, safeness yes, tell that to storm izno, he wanted to change him as well That's why they rape european girls for their look cause they look like bitches better to look like a fucking ninja as said, i give up. because white ppl dont do that at all right? google again I'll send You here and read how many of those happened by muslims in last 2 years in europe found it WHITE NINJA And do you know why are the muslims coming? Arab spring my friend don't call me your friend cuz You're nobody for me atm thats why you still capitalize the You cmon guys i see u all talking about smth u never know anything about it you* right Izno. I don't respect him . best love good best love good xD"
emotion(s)

{'emotion': 'angry',
 'probabilities': {'angry': 0.426432,
  'excited': 0.012372,
  'happy': 0.008392,
  'indifferent': 0.133715,
  'sad': 0.419089},
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [6]:
s = "are terrorists? fucking special code to detonate bomb or what what if there are other tunisian,iraq. etc great good bad on server do you think they will take it as a joke great  I fignore them lol and what do they do to offend you? they exist? they believe in killing people? you cant know what a random guy believes in if he is from iraq, he is muslim if he is muslim, he believe in killing people Trela hell no fuck trela i really need AN OFFICIAL statement thats because all christians wait till marriage to have sex. of what you just said maybe if he is member of ISIS limited go to orban and tell him they are cool and you want 1 in your house You know nothign about orban ill go to fucking orban and tell him i want one in my server k xDDDD Orban hates them Quran Quran (2:191-193) - And kill them wherever you find them, and turn them out from where they have turned you out. And Al-Fitnah [disbelief or unrest] is worse than killing... but if they desist, then lo! Allah is forgiving and merciful. And fight them until there is no more Fitnah [disbelief and worshipping of others along with Allah] and worship is for Allah alone. But if they cease, let there be no transgression except against Az-Zalimun(the polytheists, and wrong-doers, etc.) (Translation is from the Noble Quran) The verse prior to this (190) refers to fighting for the cause of Allah those who fight you leading some to claim that the entire passage refers to a defensive war in which Muslims are defending their homes and families.  The historical context of this passage is not defensive warfare, however, since Muhammad and his Muslims had just relocated to Medina and were not under attack by their Meccan adversaries.  In fact, the verses urge offensive warfare, in that Muslims are to drive Meccans out of their own city (which they later did).  Verse 190 thus means to fight those who offer resistance to Allah's rule (ie. Muslim conquest).  The use of the word persecution by some Muslim translators is disingenuous - the actual Arabic words for persecution (idtihad) - and oppression are not used instead of fitna.  Fitna can mean disbelief, or the disorder that results from unbelief or temptation.  A strict translation is 'sedition,' meaning rebellion against authority (the authority being Allah).  This is certainly what is meant in this context since the violence is explicitly commissioned until religion is for Allah - ie. unbelievers desist in their unbelief.  [Editor's note: these notes have been modified slightly after a critic misinterpreted our language. Verse 193 plainly says that 'fighting' is sanctioned even if the fitna 'ceases'.  This is about religious order, not real persecution.] have fun hitler was atheist. ANd a good painter i dont need to quote his ideology here i hope. ye, and You love them limited, so go for a conversation with him you're basically quoting bible verses here trela. Who said I love the terrorists? which are about as bad as quran ones there is a big space between hating and loving trela. I think Im out No point of this argument izno never shared his information what if hes a muslim? Trela is full of hate, so no point you still talk to him i believe those words are out of context..real muslims are friendly and not monsters..but ye this is what media presents us trela has no way of knowing my religion. read this shit book claay then simple. Quran (2:191-193) Have u ever read it? Quran (2:191-193) Quran (2:191-193) Quran (2:191-193) google it yes 3 verses thats all u got GO READ IT THEN TALK TO ME the worst guy from lebanon that i know is a fucking christian GET SOME FKN AKNOWLEDGE the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. ABOUT THIS DISEASE do you believe in god trela? Hard to not belive in yourself lol. believe* trela u didnt notes that u talk too much about muslims and say some shit about them ? i am serious do you believe in the christian god trela? do you believe in real God. christian god yes. Using those types of stories without context would be like saying that the United States is a racist/genocidal country because they bombed Japan in 1945, killing 250,000 people. The statement that they bombed Japan is true, but there's a whole context (a preceding global war that lasted 6 years) that it needs to be considered in the context of. how can I believe in 1 person which was walking on the water Izno please that was his son you dont even know christian religion how can you judge muslims so you dont believe in christian god BUT you believe in Quran which is basically an islam bible cause I don't care about this religion I don't believe in this, these retards believe that's why they kill people GEEZOS one min son of god ? wtf lol basem, as a muslim, do you belive in killing people? as a muslim k ISIS presents ISLAM in their way . and thats what u see thats what media wants us to believe. thats not even islam its disgreace im giving up on trela. he really wants to see this stuff his way, and there is no way to change his mind, no matter how much evidence to the contrary you present him. and right now he'll be convinced that we are all pro-islam becuase we sound anti-trela you see it right Izno ^ pls islam word mean security, safety, peace, belly, safeness yes, tell that to storm izno, he wanted to change him as well That's why they rape european girls for their look cause they look like bitches better to look like a fucking ninja as said, i give up. because white ppl dont do that at all right? google again I'll send You here and read how many of those happened by muslims in last 2 years in europe found it WHITE NINJA And do you know why are the muslims coming? Arab spring my friend don't call me your friend cuz You're nobody for me atm thats why you still capitalize the You cmon guys i see u all talking about smth u never know anything about it you* right Izno. I don't respect him . best love good best love good xD"
emotion(s)

{'emotion': 'indifferent',
 'probabilities': {'angry': 0.216553,
  'excited': 0.081572,
  'happy': 0.054658,
  'indifferent': 0.378342,
  'sad': 0.268874},
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [99]:
s="bad as quran ones there is a big space between hating and loving trela. I think Im out No point of this argument izno never shared his information what if hes a muslim? Trela is full of hate, so no point you still talk to him i believe those words are out of context..real muslims are friendly and not monsters..but ye this is what media presents us trela has no way of knowing my religion. read this shit book claay then simple. Quran (2:191-193) Have u ever read it? Quran (2:191-193) Quran (2:191-193) Quran (2:191-193) google it yes 3 verses thats all u got GO READ IT THEN TALK TO ME the worst guy from lebanon that i know is a fucking christian GET SOME FKN AKNOWLEDGE the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. ABOUT THIS DISEASE do you believe in god trela? Hard to not belive in yourself lol. believe* trela u didnt notes that u talk too much about muslims and say some shit about them ? i am serious do you believe in the christian god trela? do you believe in real God. christian god yes. Using those types of stories without context would be like saying that the United States is a racist/genocidal country because they bombed Japan in 1945, killing 250,000 people. The statement that they bombed Japan is true, but there's a whole context (a preceding global war that lasted 6 years) that it needs to be considered in the context of. how can I believe in 1 person which was walking on the water Izno please that was his son you dont even know christian religion how can you judge muslims so you dont believe in christian god BUT you believe in Quran which is basically an islam bible cause I don't care about this religion I don't believe in this, these retards believe that's why they kill people GEEZOS one min son of god ? wtf lol basem, as a muslim, do you belive in killing people? as a muslim k ISIS presents ISLAM in their way . and thats what u see thats what media wants us to believe. thats not even islam its disgreace im giving up on trela. he really wants to see this stuff his way, and there is no way to change his mind, no matter how much evidence to the contrary you present him. and right now he'll be convinced that we are all pro-islam becuase we sound anti-trela you see it right Izno ^ pls islam word mean security, safety, peace, belly, safeness yes, tell that to storm izno, he wanted to change him as well That's why they rape european girls for their look cause they look like bitches better to look like a fucking ninja as said, i give up. because white ppl dont do that at all right? google again I'll send You here and read how many of those happened by muslims in last 2 years in europe found it WHITE NINJA And do you know why are the muslims coming? Arab spring my friend don't call me your friend cuz You're nobody for me atm thats why you still capitalize the You cmon guys i see u all talking about smth u never know anything about it you* right Izno. I don't respect him . xD"
sentiment(s)

{'Error': 'Daily Usage Limit Exceeded.You can upgrade at https://user.apis.paralleldots.com/login'}

In [27]:
ss = "and there's nothing to change didn't used my lvl 80 against any`1. simple. and yet this conversation doesn't change anything about you being a fake friend all from me kek i tried not making a clan out of JH to prevent drama did it just make more drama? What drama? fucking all this fucking shit basem use VPN germany and dont talk no drama then :gg: >dont talk >problem solved сука блять иди нахуй пизда or that ah limited nice picture writing the truth about some1 isn't a drama *sigh* *unzips* here we go again are you playing emunow or will you be playing emulater yes it's sincerity I mean things which You can see up [5:23 PM] Trelacki: You will go cry again so what's the problem  /// not if it will be justified, i want you to get a higher lvl, but not for the sake of community wat. radek priest radek priest [5:24 PM] Trelacki: didn't used my lvl 80 against any`1.  /// you didnt, but you did say a guy is a terrorist just becausse he is from iraq which a high lvl admin shouldnt say so i need to know for god's sake trela, pls. am i getting demoted or no? Why not I'm allowed to say whatever I wnat want* that's called freedom of speech or You are like social media, You disallow this XD well, thats the reason you arent getting higher lvl, admins on jh are supposed to help people and not calling them something they might not be, whcih could be insulting to them also I didn't use word terrorist another lie iraq guy asked for help, i helped him and then you asked me why am i helping a terrorist it was vissible in cod2 channel and clay saw it hm. maybe. i believe the logic behind this is if you are higher level than usual (60+), you are suppose to be nice to people and helping them, etc. not being racist OR hurting their feelings. because if you are 60+ you are trusted with a server control therefore there are people who are trusting you. help him plant the bomb in czech republic! i was never and i never will be against you XDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD but i also dont want innocent people who want to jump to be abused Europe balances the right to free expression with the right to be free of offense, meaning your right to speak is limited by speech that I might reasonably take as offensive. if he is innocent You're not the 1 who caused You to be 200kg+ k XD OH MY GOD GUYS KEEP GOING PLEASE IM ENJOYING THIS SO MUCH np inb4 thats what she said CMON I GOR GOT do you have crusti croc? basem finished real_autist now its your turn okay im going to have dinner but when im back, i want there to still be drama ok? honestly. radek. NO OFFEENSE PLEASE I STILL LOVE YOU.  but you shouldn't be 90 IMHO because you are being mean to basem etc. which is clearly not allowed deal cu deal fu I take offense of terorrist coming to server on which I want peace basem is also mean to me sometimes and he knwos im joking and speaking some unknown language mixed with letters and numbers how do you know they are terrorists? fucking special code to detonate bomb or what what if there are other tunisian,iraq, etc. people on server do you think they will take it as a joke? I fignore them lol and what do they do to offend you? they exist? they believe in killing people? you cant know what a random guy believes in if he is from iraq, he is muslim if he is muslim, he believe in killing people Trela hell no fuck trela i really need AN OFFICIAL statement thats because all christians wait till marriage to have sex. of what you just said maybe if he is member of ISIS limited go to orban and tell him they are cool and you want 1 in your house You know nothign about orban ill go to fucking orban and tell him i want one in my server k xDDDD Orban hates them Quran Quran (2:191-193) - And kill them wherever you find them, and turn them out from where they have turned you out. And Al-Fitnah [disbelief or unrest] is worse than killing... but if they desist, then lo! Allah is forgiving and merciful. And fight them until there is no more Fitnah [disbelief and worshipping of others along with Allah] and worship is for Allah alone. But if they cease, let there be no transgression except against Az-Zalimun(the polytheists, and wrong-doers, etc.) (Translation is from the Noble Quran) The verse prior to this (190) refers to fighting for the cause of Allah those who fight you leading some to claim that the entire passage refers to a defensive war in which Muslims are defending their homes and families.  The historical context of this passage is not defensive warfare, however, since Muhammad and his Muslims had just relocated to Medina and were not under attack by their Meccan adversaries.  In fact, the verses urge offensive warfare, in that Muslims are to drive Meccans out of their own city (which they later did).  Verse 190 thus means to fight those who offer resistance to Allah's rule (ie. Muslim conquest).  The use of the word persecution by some Muslim translators is disingenuous - the actual Arabic words for persecution (idtihad) - and oppression are not used instead of fitna.  Fitna can mean disbelief, or the disorder that results from unbelief or temptation.  A strict translation is 'sedition,' meaning rebellion against authority (the authority being Allah).  This is certainly what is meant in this context since the violence is explicitly commissioned until religion is for Allah - ie. unbelievers desist in their unbelief.  [Editor's note: these notes have been modified slightly after a critic misinterpreted our language. Verse 193 plainly says that 'fighting' is sanctioned even if the fitna 'ceases'.  This is about religious order, not real persecution.] have fun hitler was atheist. ANd a good painter i dont need to quote his ideology here i hope. ye, and You love them limited, so go for a conversation with him you're basically quoting bible verses here trela. Who said I love the terrorists? which are about as bad as quran ones there is a big space between hating and loving trela. I think Im out No point of this argument izno never shared his information what if hes a muslim? Trela is full of hate, so no point you still talk to him i believe those words are out of context..real muslims are friendly and not monsters..but ye this is what media presents us trela has no way of knowing my religion. read this shit book claay then simple. Quran (2:191-193) Have u ever read it? Quran (2:191-193) Quran (2:191-193) Quran (2:191-193) google it yes 3 verses thats all u got GO READ IT THEN TALK TO ME the worst guy from lebanon that i know is a fucking christian GET SOME FKN AKNOWLEDGE the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. ABOUT THIS DISEASE do you believe in god trela? Hard to not belive in yourself lol. believe* trela u didnt notes that u talk too much about muslims and say some shit about them ? i am serious do you believe in the christian god trela? do you believe in real God. christian god yes. Using those types of stories without context would be like saying that the United States is a racist/genocidal country because they bombed Japan in 1945, killing 250,000 people. The statement that they bombed Japan is true, but there's a whole context (a preceding global war that lasted 6 years) that it needs to be considered in the context of. how can I believe in 1 person which was walking on the water Izno please that was his son you dont even know christian religion how can you judge muslims so you dont believe in christian god BUT you believe "
len(ss)

7699

In [152]:
sentiment('hello from the other side you idiotic master hello from the other side you idiotic master')

{'probabilities': {'negative': 0.548038,
  'neutral': 0.351081,
  'positive': 0.100881},
 'sentiment': 'negative',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [154]:
sentiment('hello from the other side you idiotic master hello from the other side you idiotic master hello from the other side you idiotic master hello from the other side you idiotic master')

{'probabilities': {'negative': 0.425193,
  'neutral': 0.360317,
  'positive': 0.214491},
 'sentiment': 'negative',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}